In [2]:
import sys
import os
import natsort
import numpy as np
import pickle

In [3]:
out_dir = "/home/amridul/Documents/forward_project/data/Yumi"
in_dir = "/home/amridul/Documents/forward_project/data/Yumi"

In [5]:
subjects = os.listdir(in_dir)

In [203]:
def findClosestTimeStamp(ts, timeStamps):
    lo, hi = 0, len(timeStamps)-1
    while(lo <= hi):
        mid = int((lo+hi)/2)
        if(timeStamps[mid] == ts):
            return mid
        elif (timeStamps[mid] > ts):
            hi = mid-1
        elif (timeStamps[mid] < ts):
            lo = mid+1
            
    if(lo >= len(timeStamps)):
        return hi
    if(hi < 0):
        return lo
    if(np.abs(timeStamps[lo] - ts) < np.abs(timeStamps[hi] - ts)):
        return lo
    
    return hi
    

In [221]:
def dissectKinematicSequence(sub, trial):
    fileNameLeft = sub+'_'+trial+'_yumi_left.txt'
    fileNameRight = sub+'_'+trial+'_yumi_right.txt'
    fileNameTSFrame = sub+'_'+trial+'_color_ts.txt'
    fileNameAnnotate = sub+'_'+trial+'_color_annot.txt'
    
    print(fileNameLeft, fileNameRight, fileNameTSFrame, fileNameAnnotate)
    
    frameTSArray = []
    subDir = os.path.join(in_dir, sub)
    with open(os.path.join(subDir, fileNameTSFrame)) as fTS2Frame:
        for line in fTS2Frame:
            ts = line.split('\n')[0]
            frameTSArray.append(float(ts))
            
        frameTSArray = np.array(frameTSArray)

    surgemeStartFrame = []
    surgemeEndFrame = []
    surgemeIndex = []
    surgemeSuccess = []
    dominantHand = []
    surgmemeParams = []
    segments = 0
    with open(os.path.join(subDir, fileNameAnnotate)) as fAnnotate:
        for line in fAnnotate:
            start, end, surgeme, success, domHand, param = line.split(' ')
            surgemeStartFrame.append(int(start))
            surgemeEndFrame.append(int(end))
            surgemeIndex.append(int(surgeme[1]))
            surgemeSuccess.append(bool(success))
            dominantHand.append(int(domHand))            
            surgmemeParams.append(int(param))                        
            segments += 1
    
    leftTS = []
    leftLines = open(os.path.join(subDir, fileNameLeft)).readlines()
    for line in leftLines:
        parts = line.split(',')[0]
        ts = float(parts)
        leftTS.append(ts)
        
    rightTS = []
    rightLines = open(os.path.join(subDir, fileNameRight)).readlines()
    for line in rightLines:
        parts = line.split(',')[0]
        ts = float(parts)
        rightTS.append(ts)
    
    X = []
    Y = []
    
    for frame in range(1, surgemeEndFrame[-1]+1):
        surgeme = 0
        for segment in range(segments):
            if(frame in range(surgemeStartFrame[segment], surgemeEndFrame[segment]+1)):
                surgeme = surgemeIndex[segment]
                break
        
        frameTS = frameTSArray[frame-1]

        leftTSIndex = findClosestTimeStamp(frameTS, leftTS)
        leftTimeStamp = leftTS[leftTSIndex]

        rightTSIndex = findClosestTimeStamp(frameTS, rightTS)
        rightTimeStamp = rightTS[rightTSIndex]

        leftLine = leftLines[leftTSIndex]
        rightLine = rightLines[rightTSIndex]

        leftLine = leftLine.replace("\n", '')
        leftLine = leftLine.replace("\r", '')
        leftLine = leftLine.replace(" ", '')

        rightLine = rightLine.replace("\n", '')
        rightLine = rightLine.replace("\r", '')
        rightLine = rightLine.replace(" ", '')
        
        leftParts = leftLine.split(',')
        rightParts = rightLine.split(',')
        
        leftValid = leftParts[-1]
        rightValid = rightParts[-1]

        kinData = []
        for parts in [leftParts, rightParts]:
            for part in parts[1:]:
                try:
                    data = float(part)
                    kinData.append(data)
                except:
                    if(part in ['True', 'False']):
                        kinData.append(float(bool(part)))
                    else:
                        pre, post = part.split('-')
                        data = float(pre)
                        kinData.append(data)
                        data = float(post)
                        kinData.append(data)
                        
        vecSurgeme = np.zeros(8)
        vecSurgeme[surgeme] = 1
        
        X.append(kinData)
        Y.append(vecSurgeme)
    
    return X, Y
        

In [222]:
lineNum = 0
arms = ['left', 'right']
for sub in subjects:
    trials = ['T1', 'T2', 'T3', 'T4', 'T5', 'T6']
    
    if(sub == 'S8'):
        trials = ['T1', 'T2', 'T3', 'T4']

    for trial in trials:
        X, Y = dissectKinematicSequence(sub, trial)
        fileNameX = os.path.join(os.path.join(in_dir, sub), sub+'_'+trial+'_frame_kinematic_data.pkl')
        fileNameY = os.path.join(os.path.join(in_dir, sub), sub+'_'+trial+'_frame_surgeme_data.pkl')
        
        pickle_handle = open(fileNameX, 'wb')
        pickle.dump(X, pickle_handle)
        pickle_handle.close()
        
        pickle_handle = open(fileNameY, 'wb')
        pickle.dump(Y, pickle_handle)
        pickle_handle.close()

S7_T1_yumi_left.txt S7_T1_yumi_right.txt S7_T1_color_ts.txt S7_T1_color_annot.txt
S7_T2_yumi_left.txt S7_T2_yumi_right.txt S7_T2_color_ts.txt S7_T2_color_annot.txt
S7_T3_yumi_left.txt S7_T3_yumi_right.txt S7_T3_color_ts.txt S7_T3_color_annot.txt
S7_T4_yumi_left.txt S7_T4_yumi_right.txt S7_T4_color_ts.txt S7_T4_color_annot.txt
S7_T5_yumi_left.txt S7_T5_yumi_right.txt S7_T5_color_ts.txt S7_T5_color_annot.txt
S7_T6_yumi_left.txt S7_T6_yumi_right.txt S7_T6_color_ts.txt S7_T6_color_annot.txt
S3_T1_yumi_left.txt S3_T1_yumi_right.txt S3_T1_color_ts.txt S3_T1_color_annot.txt
S3_T2_yumi_left.txt S3_T2_yumi_right.txt S3_T2_color_ts.txt S3_T2_color_annot.txt
S3_T3_yumi_left.txt S3_T3_yumi_right.txt S3_T3_color_ts.txt S3_T3_color_annot.txt
S3_T4_yumi_left.txt S3_T4_yumi_right.txt S3_T4_color_ts.txt S3_T4_color_annot.txt
S3_T5_yumi_left.txt S3_T5_yumi_right.txt S3_T5_color_ts.txt S3_T5_color_annot.txt
S3_T6_yumi_left.txt S3_T6_yumi_right.txt S3_T6_color_ts.txt S3_T6_color_annot.txt
S8_T1_yumi_left.